In [1]:
from theano.sandbox import cuda
#cuda.use('gpu2')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 950 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5110)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


## Setup

In [3]:
batch_size=64

In [4]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [5]:
X_test = np.expand_dims(X_test,1)
X_train = np.expand_dims(X_train,1) # (60000, 28, 28) => (600000, 1, 28, 28)

In [6]:
X_train.shape

(60000, 1, 28, 28)

In [7]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [8]:
y_train = onehot(y_train)
y_test = onehot(y_test)

In [9]:
y_train[:5]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [10]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

In [11]:
def norm_input(x): return (x-mean_px)/std_px

## Linear model

In [12]:
def get_lin_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [13]:
lm = get_lin_model()

/home/tsuyoshi/.pyenv/versions/anaconda3-4.3.0/envs/py2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [16]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [16]:
# http://forums.fast.ai/t/lesson-2-fit-model/1143/5

# keras1.1.0
#lm.fit_generator(batches, batches.N, nb_epoch=1, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N)

# keras1.2.2
lm.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 14s - loss: 0.4236 - acc: 0.8743 - val_loss: 0.3017 - val_acc: 0.9147


In [17]:
lm.optimizer.lr=0.1

In [18]:
#lm.fit_generator(batches, batches.N, nb_epoch=1, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
lm.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/1
60000/60000 [==============================] - 14s - loss: 0.2998 - acc: 0.9153 - val_loss: 0.2810 - val_acc: 0.9224


In [19]:
lm.optimizer.lr=0.01

In [20]:
#lm.fit_generator(batches, batches.N, nb_epoch=4, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
lm.fit_generator(batches, batches.n, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/4
60000/60000 [==============================] - 14s - loss: 0.2847 - acc: 0.9199 - val_loss: 0.2964 - val_acc: 0.9141
Epoch 2/4
60000/60000 [==============================] - 14s - loss: 0.2794 - acc: 0.9218 - val_loss: 0.2735 - val_acc: 0.9220
Epoch 3/4
60000/60000 [==============================] - 14s - loss: 0.2721 - acc: 0.9238 - val_loss: 0.2952 - val_acc: 0.9189
Epoch 4/4
60000/60000 [==============================] - 14s - loss: 0.2692 - acc: 0.9250 - val_loss: 0.2784 - val_acc: 0.9220


## Single dense layer

In [13]:
#def get_fc_model():
#    model = Sequential([
#        Lambda(norm_input, input_shape=(1,28,28)),
#        Flatten(),
#        Dense(512, activation='softmax'),
#        Dense(10, activation='softmax')
#        ])
#    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
#    return model

def get_fc_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(512, activation='relu'), # relu
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [14]:
fc = get_fc_model()

/home/tsuyoshi/.pyenv/versions/anaconda3-4.3.0/envs/py2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [17]:
#fc.fit_generator(batches, batches.N, nb_epoch=1, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
fc.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/1
60000/60000 [==============================] - 14s - loss: 0.1913 - acc: 0.9419 - val_loss: 0.0930 - val_acc: 0.9713


In [18]:
fc.optimizer.lr=0.1

In [19]:
#fc.fit_generator(batches, batches.N, nb_epoch=4, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
fc.fit_generator(batches, batches.n, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/4
60000/60000 [==============================] - 14s - loss: 0.0807 - acc: 0.9753 - val_loss: 0.0810 - val_acc: 0.9756
Epoch 2/4
60000/60000 [==============================] - 14s - loss: 0.0557 - acc: 0.9822 - val_loss: 0.0649 - val_acc: 0.9789
Epoch 3/4
60000/60000 [==============================] - 14s - loss: 0.0411 - acc: 0.9864 - val_loss: 0.0638 - val_acc: 0.9823
Epoch 4/4
60000/60000 [==============================] - 14s - loss: 0.0322 - acc: 0.9895 - val_loss: 0.0766 - val_acc: 0.9778


In [20]:
fc.optimizer.lr=0.01

In [21]:
#fc.fit_generator(batches, batches.N, nb_epoch=4, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
fc.fit_generator(batches, batches.n, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/4
60000/60000 [==============================] - 14s - loss: 0.0266 - acc: 0.9908 - val_loss: 0.0743 - val_acc: 0.9786
Epoch 2/4
60000/60000 [==============================] - 14s - loss: 0.0246 - acc: 0.9914 - val_loss: 0.0816 - val_acc: 0.9766
Epoch 3/4
60000/60000 [==============================] - 14s - loss: 0.0210 - acc: 0.9927 - val_loss: 0.0978 - val_acc: 0.9779
Epoch 4/4
60000/60000 [==============================] - 14s - loss: 0.0189 - acc: 0.9938 - val_loss: 0.0916 - val_acc: 0.9782


## Basic 'VGG-style' CNN

In [28]:
def get_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy']) # lr=0.001
    return model

In [29]:
model = get_model()

/home/tsuyoshi/.pyenv/versions/anaconda3-4.3.0/envs/py2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_3 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [30]:
#model.fit_generator(batches, batches.N, nb_epoch=1, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/1
60000/60000 [==============================] - 15s - loss: 0.1146 - acc: 0.9636 - val_loss: 0.0312 - val_acc: 0.9883


In [31]:
model.optimizer.lr=0.1

In [32]:
#model.fit_generator(batches, batches.N, nb_epoch=1, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/1
60000/60000 [==============================] - 15s - loss: 0.0358 - acc: 0.9895 - val_loss: 0.0339 - val_acc: 0.9885


In [33]:
model.optimizer.lr=0.01

In [34]:
#model.fit_generator(batches, batches.N, nb_epoch=8, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=8, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/8
60000/60000 [==============================] - 15s - loss: 0.0256 - acc: 0.9920 - val_loss: 0.0277 - val_acc: 0.9918
Epoch 2/8
60000/60000 [==============================] - 15s - loss: 0.0185 - acc: 0.9943 - val_loss: 0.0314 - val_acc: 0.9909
Epoch 3/8
60000/60000 [==============================] - 15s - loss: 0.0165 - acc: 0.9943 - val_loss: 0.0247 - val_acc: 0.9924
Epoch 4/8
60000/60000 [==============================] - 15s - loss: 0.0134 - acc: 0.9957 - val_loss: 0.0257 - val_acc: 0.9931
Epoch 5/8
60000/60000 [==============================] - 15s - loss: 0.0117 - acc: 0.9963 - val_loss: 0.0195 - val_acc: 0.9949
Epoch 6/8
60000/60000 [==============================] - 15s - loss: 0.0086 - acc: 0.9974 - val_loss: 0.0298 - val_acc: 0.9910
Epoch 7/8
60000/60000 [==============================] - 15s - loss: 0.0096 - acc: 0.9970 - val_loss: 0.0282 - val_acc: 0.9929
Epoch 8/8
60000/60000 [==============================] - 15s - loss: 0.0073 - acc: 0.9979 - val_loss: 0.0281 - 

## Data augmentation

In [35]:
model = get_model()

/home/tsuyoshi/.pyenv/versions/anaconda3-4.3.0/envs/py2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_4 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [36]:
gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(X_train, y_train, batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [37]:
#model.fit_generator(batches, batches.N, nb_epoch=1, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/1
60000/60000 [==============================] - 16s - loss: 0.2016 - acc: 0.9371 - val_loss: 0.0618 - val_acc: 0.9804


In [38]:
model.optimizer.lr=0.1

In [39]:
#model.fit_generator(batches, batches.N, nb_epoch=4, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/4
60000/60000 [==============================] - 16s - loss: 0.0686 - acc: 0.9776 - val_loss: 0.0636 - val_acc: 0.9799
Epoch 2/4
60000/60000 [==============================] - 16s - loss: 0.0551 - acc: 0.9829 - val_loss: 0.0462 - val_acc: 0.9861
Epoch 3/4
60000/60000 [==============================] - 16s - loss: 0.0485 - acc: 0.9853 - val_loss: 0.0392 - val_acc: 0.9881
Epoch 4/4
60000/60000 [==============================] - 16s - loss: 0.0422 - acc: 0.9866 - val_loss: 0.0387 - val_acc: 0.9889


In [40]:
model.optimizer.lr=0.01

In [41]:
#model.fit_generator(batches, batches.N, nb_epoch=8, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=8, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/8
60000/60000 [==============================] - 16s - loss: 0.0383 - acc: 0.9883 - val_loss: 0.0402 - val_acc: 0.9868
Epoch 2/8
60000/60000 [==============================] - 16s - loss: 0.0381 - acc: 0.9877 - val_loss: 0.0336 - val_acc: 0.9892
Epoch 3/8
60000/60000 [==============================] - 16s - loss: 0.0344 - acc: 0.9892 - val_loss: 0.0351 - val_acc: 0.9882
Epoch 4/8
60000/60000 [==============================] - 16s - loss: 0.0332 - acc: 0.9897 - val_loss: 0.0303 - val_acc: 0.9898
Epoch 5/8
60000/60000 [==============================] - 16s - loss: 0.0315 - acc: 0.9898 - val_loss: 0.0267 - val_acc: 0.9905
Epoch 6/8
60000/60000 [==============================] - 16s - loss: 0.0291 - acc: 0.9909 - val_loss: 0.0300 - val_acc: 0.9911
Epoch 7/8
60000/60000 [==============================] - 16s - loss: 0.0286 - acc: 0.9910 - val_loss: 0.0332 - val_acc: 0.9892
Epoch 8/8
60000/60000 [==============================] - 16s - loss: 0.0257 - acc: 0.9919 - val_loss: 0.0365 - 

In [42]:
model.optimizer.lr=0.001

In [43]:
#model.fit_generator(batches, batches.N, nb_epoch=14, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=14, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/14
60000/60000 [==============================] - 16s - loss: 0.0260 - acc: 0.9923 - val_loss: 0.0293 - val_acc: 0.9908
Epoch 2/14
60000/60000 [==============================] - 16s - loss: 0.0251 - acc: 0.9921 - val_loss: 0.0333 - val_acc: 0.9906
Epoch 3/14
60000/60000 [==============================] - 16s - loss: 0.0240 - acc: 0.9922 - val_loss: 0.0306 - val_acc: 0.9911
Epoch 4/14
60000/60000 [==============================] - 16s - loss: 0.0253 - acc: 0.9920 - val_loss: 0.0367 - val_acc: 0.9896
Epoch 5/14
60000/60000 [==============================] - 16s - loss: 0.0225 - acc: 0.9928 - val_loss: 0.0273 - val_acc: 0.9917
Epoch 6/14
60000/60000 [==============================] - 16s - loss: 0.0235 - acc: 0.9927 - val_loss: 0.0303 - val_acc: 0.9904
Epoch 7/14
60000/60000 [==============================] - 16s - loss: 0.0223 - acc: 0.9931 - val_loss: 0.0306 - val_acc: 0.9894
Epoch 8/14
60000/60000 [==============================] - 16s - loss: 0.0222 - acc: 0.9931 - val_loss: 0

In [44]:
model.optimizer.lr=0.0001

In [45]:
#model.fit_generator(batches, batches.N, nb_epoch=10, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=10, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/10
60000/60000 [==============================] - 16s - loss: 0.0194 - acc: 0.9940 - val_loss: 0.0278 - val_acc: 0.9920
Epoch 2/10
60000/60000 [==============================] - 16s - loss: 0.0189 - acc: 0.9943 - val_loss: 0.0294 - val_acc: 0.9914
Epoch 3/10
60000/60000 [==============================] - 16s - loss: 0.0202 - acc: 0.9937 - val_loss: 0.0270 - val_acc: 0.9921
Epoch 4/10
60000/60000 [==============================] - 16s - loss: 0.0188 - acc: 0.9939 - val_loss: 0.0308 - val_acc: 0.9913
Epoch 5/10
60000/60000 [==============================] - 16s - loss: 0.0187 - acc: 0.9941 - val_loss: 0.0270 - val_acc: 0.9925
Epoch 6/10
60000/60000 [==============================] - 16s - loss: 0.0178 - acc: 0.9945 - val_loss: 0.0278 - val_acc: 0.9904
Epoch 7/10
60000/60000 [==============================] - 16s - loss: 0.0178 - acc: 0.9944 - val_loss: 0.0282 - val_acc: 0.9918
Epoch 8/10
60000/60000 [==============================] - 16s - loss: 0.0187 - acc: 0.9939 - val_loss: 0

## Batchnorm + data augmentation

In [46]:
# https://keras.io/ja/layers/normalization/
# axis: 整数．正規化する軸（典型的には，特徴量の軸）．例えば，data_format="channels_first"のConv2Dの後では，axis=1にします
# axis=0 -> first dimension
# axis=-1 -> last dimension

def get_model_bn():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1), 
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(), # axis=-1
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [47]:
model = get_model_bn()

/home/tsuyoshi/.pyenv/versions/anaconda3-4.3.0/envs/py2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_5 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [48]:
#model.fit_generator(batches, batches.N, nb_epoch=1, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/1
60000/60000 [==============================] - 19s - loss: 0.1633 - acc: 0.9492 - val_loss: 0.0703 - val_acc: 0.9785


In [49]:
model.optimizer.lr=0.1

In [50]:
#model.fit_generator(batches, batches.N, nb_epoch=4, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/4
60000/60000 [==============================] - 19s - loss: 0.0737 - acc: 0.9765 - val_loss: 0.0513 - val_acc: 0.9820
Epoch 2/4
60000/60000 [==============================] - 19s - loss: 0.0603 - acc: 0.9812 - val_loss: 0.0426 - val_acc: 0.9858
Epoch 3/4
60000/60000 [==============================] - 18s - loss: 0.0532 - acc: 0.9833 - val_loss: 0.0431 - val_acc: 0.9863
Epoch 4/4
60000/60000 [==============================] - 18s - loss: 0.0488 - acc: 0.9845 - val_loss: 0.0462 - val_acc: 0.9838


In [51]:
model.optimizer.lr=0.01

In [52]:
#model.fit_generator(batches, batches.N, nb_epoch=12, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=12, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/12
60000/60000 [==============================] - 18s - loss: 0.0452 - acc: 0.9861 - val_loss: 0.0367 - val_acc: 0.9884
Epoch 2/12
60000/60000 [==============================] - 18s - loss: 0.0427 - acc: 0.9868 - val_loss: 0.0383 - val_acc: 0.9886
Epoch 3/12
60000/60000 [==============================] - 18s - loss: 0.0382 - acc: 0.9881 - val_loss: 0.0321 - val_acc: 0.9899
Epoch 4/12
60000/60000 [==============================] - 18s - loss: 0.0387 - acc: 0.9880 - val_loss: 0.0316 - val_acc: 0.9888
Epoch 5/12
60000/60000 [==============================] - 18s - loss: 0.0353 - acc: 0.9889 - val_loss: 0.0288 - val_acc: 0.9901
Epoch 6/12
60000/60000 [==============================] - 18s - loss: 0.0329 - acc: 0.9894 - val_loss: 0.0283 - val_acc: 0.9919
Epoch 7/12
60000/60000 [==============================] - 18s - loss: 0.0319 - acc: 0.9902 - val_loss: 0.0249 - val_acc: 0.9919
Epoch 8/12
60000/60000 [==============================] - 18s - loss: 0.0303 - acc: 0.9904 - val_loss: 0

In [53]:
model.optimizer.lr=0.001

In [54]:
#model.fit_generator(batches, batches.N, nb_epoch=12, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=12, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/12
60000/60000 [==============================] - 18s - loss: 0.0245 - acc: 0.9923 - val_loss: 0.0227 - val_acc: 0.9933
Epoch 2/12
60000/60000 [==============================] - 18s - loss: 0.0233 - acc: 0.9929 - val_loss: 0.0277 - val_acc: 0.9904
Epoch 3/12
60000/60000 [==============================] - 18s - loss: 0.0230 - acc: 0.9927 - val_loss: 0.0271 - val_acc: 0.9923
Epoch 4/12
60000/60000 [==============================] - 18s - loss: 0.0243 - acc: 0.9923 - val_loss: 0.0234 - val_acc: 0.9925
Epoch 5/12
60000/60000 [==============================] - 18s - loss: 0.0232 - acc: 0.9929 - val_loss: 0.0216 - val_acc: 0.9923
Epoch 6/12
60000/60000 [==============================] - 18s - loss: 0.0213 - acc: 0.9930 - val_loss: 0.0225 - val_acc: 0.9928
Epoch 7/12
60000/60000 [==============================] - 18s - loss: 0.0206 - acc: 0.9932 - val_loss: 0.0266 - val_acc: 0.9925
Epoch 8/12
60000/60000 [==============================] - 18s - loss: 0.0219 - acc: 0.9928 - val_loss: 0

## Batchnorm + dropout + data augmentation

In [55]:
def get_model_bn_do():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [56]:
model = get_model_bn_do()

/home/tsuyoshi/.pyenv/versions/anaconda3-4.3.0/envs/py2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_6 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [57]:
#model.fit_generator(batches, batches.N, nb_epoch=1, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/1
60000/60000 [==============================] - 19s - loss: 0.2258 - acc: 0.9318 - val_loss: 0.0704 - val_acc: 0.9784


In [58]:
model.optimizer.lr=0.1

In [59]:
#model.fit_generator(batches, batches.N, nb_epoch=4, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/4
60000/60000 [==============================] - 19s - loss: 0.0873 - acc: 0.9730 - val_loss: 0.0641 - val_acc: 0.9802
Epoch 2/4
60000/60000 [==============================] - 19s - loss: 0.0755 - acc: 0.9765 - val_loss: 0.0460 - val_acc: 0.9848
Epoch 3/4
60000/60000 [==============================] - 18s - loss: 0.0654 - acc: 0.9799 - val_loss: 0.0477 - val_acc: 0.9847
Epoch 4/4
60000/60000 [==============================] - 19s - loss: 0.0589 - acc: 0.9820 - val_loss: 0.0357 - val_acc: 0.9889


In [60]:
model.optimizer.lr=0.01

In [61]:
#model.fit_generator(batches, batches.N, nb_epoch=12, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=12, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/12
60000/60000 [==============================] - 19s - loss: 0.0602 - acc: 0.9819 - val_loss: 0.0385 - val_acc: 0.9872
Epoch 2/12
60000/60000 [==============================] - 19s - loss: 0.0547 - acc: 0.9834 - val_loss: 0.0401 - val_acc: 0.9873
Epoch 3/12
60000/60000 [==============================] - 19s - loss: 0.0524 - acc: 0.9842 - val_loss: 0.0362 - val_acc: 0.9884
Epoch 4/12
60000/60000 [==============================] - 19s - loss: 0.0490 - acc: 0.9855 - val_loss: 0.0253 - val_acc: 0.9925
Epoch 5/12
60000/60000 [==============================] - 19s - loss: 0.0456 - acc: 0.9859 - val_loss: 0.0249 - val_acc: 0.9915
Epoch 6/12
60000/60000 [==============================] - 19s - loss: 0.0439 - acc: 0.9871 - val_loss: 0.0349 - val_acc: 0.9888
Epoch 7/12
60000/60000 [==============================] - 19s - loss: 0.0420 - acc: 0.9872 - val_loss: 0.0298 - val_acc: 0.9900
Epoch 8/12
60000/60000 [==============================] - 18s - loss: 0.0421 - acc: 0.9876 - val_loss: 0

In [62]:
model.optimizer.lr=0.001

In [63]:
#model.fit_generator(batches, batches.N, nb_epoch=1, 
#                    validation_data=test_batches, nb_val_samples=test_batches.N) # keras1.1.0
model.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) # keras1.2.2

Epoch 1/1
60000/60000 [==============================] - 18s - loss: 0.0339 - acc: 0.9893 - val_loss: 0.0240 - val_acc: 0.9923


## Ensembling

In [65]:
def fit_model():
    model = get_model_bn_do()
#    model.fit_generator(batches, batches.N, nb_epoch=1, verbose=0,
#                        validation_data=test_batches, nb_val_samples=test_batches.N)
    model.fit_generator(batches, batches.n, nb_epoch=1, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.n)    
    model.optimizer.lr=0.1
#    model.fit_generator(batches, batches.N, nb_epoch=4, verbose=0,
#                        validation_data=test_batches, nb_val_samples=test_batches.N)
    model.fit_generator(batches, batches.n, nb_epoch=4, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.n)    
    model.optimizer.lr=0.01
#    model.fit_generator(batches, batches.N, nb_epoch=12, verbose=0,
#                        validation_data=test_batches, nb_val_samples=test_batches.N)
    model.fit_generator(batches, batches.n, nb_epoch=12, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.n)    
    model.optimizer.lr=0.001
#    model.fit_generator(batches, batches.N, nb_epoch=18, verbose=0,
#                        validation_data=test_batches, nb_val_samples=test_batches.N)
    model.fit_generator(batches, batches.n, nb_epoch=18, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.n)    
    return model

In [66]:
models = [fit_model() for i in range(6)]

/home/tsuyoshi/.pyenv/versions/anaconda3-4.3.0/envs/py2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_7 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))
/home/tsuyoshi/.pyenv/versions/anaconda3-4.3.0/envs/py2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_8 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))
/home/tsuyoshi/.pyenv/versions/anaconda3-4.3.0/envs/py2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_s

In [67]:
path = "data/mnist/"
model_path = path + 'models/'

In [68]:
for i,m in enumerate(models):
    m.save_weights(model_path+'cnn-mnist23-'+str(i)+'.pkl')

In [69]:
evals = np.array([m.evaluate(X_test, y_test, batch_size=256) for m in models]) # (6, 2)

 9216/10000 [==========================>...] - ETA: 0s

In [71]:
evals.mean(axis=0)

array([ 0.0137,  0.9958])

In [72]:
all_preds = np.stack([m.predict(X_test, batch_size=256) for m in models]) # (6, 10000, 10)

In [71]:
all_preds.shape

(6, 10000, 10)

In [73]:
avg_preds = all_preds.mean(axis=0) # (10000, 10)

In [75]:
keras.metrics.categorical_accuracy(y_test, avg_preds).eval()

array(0.9969000220298767, dtype=float32)

| Model | Train loss | Train accuracy | Test loss | Test accuracy |
|:-----------|------------|------------|------------|------------|
| Linear model | 0.4236 | 0.8743 | 0.3017 | 0.9147 |
| Single dense layer  |  0.0189  | 0.9938 | 0.0916 | 0.9782 |
| Basic 'VGG-style' CNN | 0.0073 | 0.9979 | 0.0281| 0.9935 |
| Data augmentation | 0.0177 | 0.9949 | 0.0255 | 0.9925 |
| Batchnorm + data augmentation | 0.0189 | 0.9937 | 0.0265 | 0.9907 |
| Batchnorm + dropout + data augmentation | 0.0339| 0.9893 | 0.0240 | 0.9923 |
| Ensembling (Batchnorm + dropout + data augmentation) | | | | 0.9969000220298767 |